In [1]:
from qiboml.models.encoding_decoding import *
from qiboml.models.ansatze import *

model = [BinaryEncodingLayer(nqubits=3), ReuploadingLayer(nqubits=3), QuantumDecodingLayer(nqubits=3)]
for m in model:
    print(m)

BinaryEncodingLayer(nqubits=3, qubits=range(0, 3), circuit=<qibo.models.circuit.Circuit object at 0x795ab0567f40>, initial_state=None, backend=tensorflow)
ReuploadingLayer(nqubits=3, qubits=range(0, 3), circuit=<qibo.models.circuit.Circuit object at 0x795a052328f0>, initial_state=None, backend=tensorflow)
QuantumDecodingLayer(nqubits=3, qubits=range(0, 3), circuit=<qibo.models.circuit.Circuit object at 0x795a052335e0>, initial_state=None, backend=tensorflow, nshots=1000)


In [3]:
import numpy as np

data = np.array([[0,0,0], [0,1,0], [1,1,0]])
result = []

for d in data:
    x = model[0].forward(d)
    x = model[1].forward(x)
    result.append(model[2].forward(x).probabilities())

print(result)

[<tf.Tensor: shape=(8,), dtype=float64, numpy=array([1., 0., 0., 0., 0., 0., 0., 0.])>, <tf.Tensor: shape=(8,), dtype=float64, numpy=array([0., 0., 1., 0., 0., 0., 0., 0.])>, <tf.Tensor: shape=(8,), dtype=float64, numpy=array([0., 0., 0., 0., 1., 0., 0., 0.])>]
